# Load the dividing point of videos
Dividing Point is the timestamp when the news reporters disappear in the scene.

In [1]:
import pandas as pd

In [ ]:
cutpt_df = pd.read_pickle(r"./icable-clip-feature-extraction/cut_points.pkl")
cutpt_df = cutpt_df[cutpt_df["has_problem"] == False]

In [9]:
print(cutpt_df)

      video_name  cut_frame  has_problem
0          0.mp4        197        False
1          1.mp4         38        False
2       1000.mp4        359        False
3      10001.mp4        347        False
4      10002.mp4        143        False
...          ...        ...          ...
28894   9991.mp4        199        False
28895   9992.mp4        326        False
28896   9994.mp4        158        False
28897   9996.mp4        132        False
28898   9997.mp4        296        False

[28895 rows x 3 columns]


# Generating transcribing as SRT files and output audio stream as mp3 files 

In [ ]:
import os
import cv2
import subprocess
import pandas as pd
import whisper
from whisper.utils import get_writer
from tqdm import tqdm  


model = whisper.load_model("large-v3")  


video_folder = r"./icable Reporter Videos"


output_dir = "./srts"
os.makedirs(output_dir, exist_ok=True)


mp3_output_dir = r"./icable Reporter MP3"
os.makedirs(mp3_output_dir, exist_ok=True)

def get_frame_rate(video_path):
    """
    Get the frame rate of a video using OpenCV.
    """
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        return None
    frame_rate = cap.get(cv2.CAP_PROP_FPS)
    cap.release()
    return frame_rate


for index, row in tqdm(cutpt_df.iterrows(), total=len(cutpt_df), desc="Processing videos"):
    video_name = row["video_name"]
    cut_frame = row["cut_frame"]
    video_path = os.path.join(video_folder, video_name)

    if row["has_problem"]:
        print(f"Skipping {video_name} due to problems.")
        continue

    

    
    frame_rate = get_frame_rate(video_path)
    if frame_rate is None:
        print(f"Could not determine frame rate for {video_name}. Skipping.")
        continue

    
    duration_seconds = cut_frame / frame_rate

    
    mp3_file_path = os.path.join(mp3_output_dir, f"{os.path.splitext(video_name)[0]}.mp3")

    
    ffmpeg_cmd = [
        "ffmpeg",
        "-y",  
        "-i", video_path,  
        "-t", str(duration_seconds),  
        "-vn",  
        "-c:a", "libmp3lame",  
        mp3_file_path,  
    ]

    try:
        subprocess.run(ffmpeg_cmd, check=True)
    except subprocess.CalledProcessError as e:
        print(f"Error processing {video_name}: {e}")
        continue

    
    try:
        result = model.transcribe(mp3_file_path, task="transcribe", language="cantonese", fp16=False)

        
        writer = get_writer("srt", output_dir)
        writer(result, os.path.join(output_dir, f"{os.path.splitext(video_name)[0]}.srt"))
        
    except Exception as e:
        print(f"Error transcribing {video_name}: {e}")
        continue

    
    


C:\Users\user\AppData\Local\anaconda3\Lib\site-packages\whisper\__init__.py:150: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(fp, map_location=devic

Error transcribing 28964.mp4: Failed to load audio: ffmpeg version 7.1-full_build-www.gyan.dev Copyright (c) 2000-2024 the FFmpeg developers
  built with gcc 14.2.0 (Rev1, Built by MSYS2 project)
  configuration: --enable-gpl --enable-version3 --enable-static --disable-w32threads --disable-autodetect --enable-fontconfig --enable-iconv --enable-gnutls --enable-libxml2 --enable-gmp --enable-bzlib --enable-lzma --enable-libsnappy --enable-zlib --enable-librist --enable-libsrt --enable-libssh --enable-libzmq --enable-avisynth --enable-libbluray --enable-libcaca --enable-sdl2 --enable-libaribb24 --enable-libaribcaption --enable-libdav1d --enable-libdavs2 --enable-libopenjpeg --enable-libquirc --enable-libuavs3d --enable-libxevd --enable-libzvbi --enable-libqrencode --enable-librav1e --enable-libsvtav1 --enable-libvvenc --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxavs2 --enable-libxeve --enable-libxvid --enable-libaom --enable-libjxl --enable-libvpx --enable-mediafoundatio

Processing videos: 100%|██████████████████████████████████████████████████████| 28895/28895 [13:28:03<00:00,  1.68s/it]
